# Lecture 2: Introduction to Feature Engineering

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [2]:
articles = pd.read_csv('../data/articles.csv')
customers = pd.read_csv('../data/customers.csv')
sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

## The H&M Dataset

In [3]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [4]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [7]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05), ("10", 0.1)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05
0.1


In [8]:
articles_sample = pd.read_csv('../data/articles_sample1.csv.gz')
customers_sample = pd.read_csv('../data/customers_sample1.csv.gz')
transactions_sample = pd.read_csv('../data/transactions_sample1.csv.gz')

In [9]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13719 entries, 0 to 13718
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             13719 non-null  object 
 1   FN                      4776 non-null   float64
 2   Active                  4632 non-null   float64
 3   club_member_status      13664 non-null  object 
 4   fashion_news_frequency  13568 non-null  object 
 5   age                     13573 non-null  float64
 6   postal_code             13719 non-null  object 
dtypes: float64(3), object(4)
memory usage: 750.4+ KB


In [10]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325780 entries, 0 to 325779
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   t_dat             325780 non-null  object 
 1   customer_id       325780 non-null  object 
 2   article_id        325780 non-null  int64  
 3   price             325780 non-null  float64
 4   sales_channel_id  325780 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 12.4+ MB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [11]:
# If you'd rather use a sample. Uncomment the following code:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [12]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [13]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,00a017d6e1010e117e3881be44027688795bf5f3c36369...,610776024,0.008458,1,1
1,2018-09-20,00a017d6e1010e117e3881be44027688795bf5f3c36369...,186262006,0.004220,1,1
2,2018-09-20,00a017d6e1010e117e3881be44027688795bf5f3c36369...,610776023,0.008458,1,1
3,2018-09-20,00a4f2c8cc87ffb0ea7db47b4b0b247b9ef11033c8d976...,610097001,0.028797,2,1
4,2018-09-20,0491c53266f9cbbebf462916afa25c791ee4b84bb94633...,573716010,0.025407,1,1


In [14]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [15]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,00a017d6e1010e117e3881be44027688795bf5f3c36369...,610776024,0.008458,1,1
1,2018-09-20,00a017d6e1010e117e3881be44027688795bf5f3c36369...,186262006,0.004220,1,1
2,2018-09-20,00a017d6e1010e117e3881be44027688795bf5f3c36369...,610776023,0.008458,1,1
3,2018-09-20,00a4f2c8cc87ffb0ea7db47b4b0b247b9ef11033c8d976...,610097001,0.028797,2,1
4,2018-09-20,0491c53266f9cbbebf462916afa25c791ee4b84bb94633...,573716010,0.025407,1,1


In [16]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [17]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

325780


In [18]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [19]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [20]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2019-05-13,98fb70b33aeed8a6532d3189fbb5beed1bfb20cacf4692...,841298004,0.032136,2,0
1,2019-06-15,87b2b92bc684a02420a14f72053a08a20df41a20544be2...,805395001,0.037712,1,0
2,2020-02-08,551ba94cc3b8a2b1cb5cc6580e689e517cb769d255b202...,747807002,0.025407,1,0
3,2019-03-23,f9477529fa46fbecd9901116b6cced336ea5bdf11a5032...,773375001,0.041559,1,0
4,2020-05-12,32be3b8b59e7015b8c4da0c7c76625262e4f282a266bae...,738881002,0.015237,1,0


In [21]:
neg_transactions.shape

(358358, 6)

In [22]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [23]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [24]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651560 entries, 0 to 651559
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   t_dat                         651560 non-null  object 
 1   customer_id                   651560 non-null  object 
 2   article_id                    651560 non-null  object 
 3   price                         651560 non-null  object 
 4   sales_channel_id              651560 non-null  object 
 5   ordered                       651560 non-null  object 
 6   FN                            249231 non-null  float64
 7   Active                        243502 non-null  float64
 8   club_member_status            649671 non-null  object 
 9   fashion_news_frequency        646776 non-null  object 
 10  age                           646793 non-null  float64
 11  postal_code                   651560 non-null  object 
 12  product_code                  651560 non-nul

### Basic Preprocessing
Some very basic preprocessing.

In [25]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,00a017d6e1010e117e3881be44027688795bf5f3c36369...,25.0,610776024,1,0.008458,1
1,3ce52143471d282eb882fc4869e900b8e5e6f53ca99adf...,28.0,610776024,2,0.008458,1
2,b49c804654e2bbf33c9c6d7be8f91d02b7c759c68d8bbe...,23.0,610776024,2,0.008458,0
3,5dac2574475af546fd33bd0fc25752878faf5e3a91bd4a...,23.0,610776024,2,0.008458,0
4,83366e822f3d1b09270fd862649261a3a8ed63e3217f5e...,25.0,610776024,1,0.008458,1


In [26]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [27]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [28]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['000934651054f08396856cd83fad3b36b97ab95a0baf79cf86769e92bf3fef40'],
      dtype=object)

In [29]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,30,25.0,9356,1,0.008458,1
1,3192,28.0,9356,2,0.008458,1
2,9586,23.0,9356,2,0.008458,0
3,4902,23.0,9356,2,0.008458,0
4,6914,25.0,9356,1,0.008458,1


In [30]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [31]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [32]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,30,25.0,9356,0.008458,1,1,0
1,3192,28.0,9356,0.008458,1,0,1
2,9586,23.0,9356,0.008458,0,0,1
3,4902,23.0,9356,0.008458,0,0,1
4,6914,25.0,9356,0.008458,1,1,0


In [33]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [34]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
513302,3970,21.0,26944,0.022017,0,1
200644,10190,23.0,47460,0.022508,0,1
70917,4616,30.0,4421,0.025407,0,1
169554,7357,48.0,32888,0.023712,0,1
614107,11096,27.0,23795,0.022017,0,1


In [35]:
y_train.head()

513302    1
200644    0
70917     1
169554    0
614107    1
Name: ordered, dtype: int64

## Baseline Model

In [36]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [37]:
baseline.predict_proba(X_test)

array([[0.48641632, 0.51358368],
       [0.4879349 , 0.5120651 ],
       [0.50741811, 0.49258189],
       ...,
       [0.49958818, 0.50041182],
       [0.50211495, 0.49788505],
       [0.52035248, 0.47964752]])

In [38]:
y_test

616298    0
132740    0
283045    0
456216    1
118344    1
         ..
188339    0
137938    1
190708    0
193842    0
233875    0
Name: ordered, Length: 65156, dtype: int64

In [39]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.5144576094296764

In [40]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.58      0.54     32469
           1       0.52      0.45      0.48     32687

    accuracy                           0.51     65156
   macro avg       0.51      0.51      0.51     65156
weighted avg       0.51      0.51      0.51     65156



In [41]:
predictions

array([1, 1, 0, ..., 1, 0, 0])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).


### Sold last 6 months ###
For every transaction entry, track how much this article has been sold in the last 6 months. Note: This could've been done much more efficiently, but I will keep it as is to show my earlier approach.

In [42]:
# compute for each article how much it has been sold in the last 6 months
# first get articles and dates where ordered == 1
articles_by_date = transactions.loc[transactions["ordered"] == 1][["article_id", "t_dat"]]
articles_by_date["t_dat"] = pd.to_datetime(articles_by_date["t_dat"])
most_recent = articles_by_date["t_dat"].max()
articles_by_date["months_passed"] = 12 * (most_recent.year - articles_by_date["t_dat"].dt.year) + (most_recent.month - articles_by_date["t_dat"].dt.month)
articles_by_date.head()

,article_id,t_dat,months_passed
0,610776024,2018-09-20,24
1,610776024,2018-09-20,24
4,610776024,2019-01-03,20
5,610776024,2019-04-20,17
7,610776024,2019-07-28,14


In [43]:
# retrieve how much every article has been sold in last six months
recently_sold = articles_by_date.loc[articles_by_date["months_passed"] <= 6]
recently_sold = recently_sold["article_id"].value_counts().rename_axis("article_id").reset_index(name="count")

Now that for every article there is a count of how much it has been sold in the last six months, the transactions dataframe can be created. The count is added by performing an inner join on the article id. For articles that haven't been sold in the last six months, a 0 entry will be added later.

In [44]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed = transactions_processed.merge(recently_sold, how="inner", on='article_id')
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,count
0,00a017d6e1010e117e3881be44027688795bf5f3c36369...,25.0,610776024,1,0.008458,1,2
1,3ce52143471d282eb882fc4869e900b8e5e6f53ca99adf...,28.0,610776024,2,0.008458,1,2
2,b49c804654e2bbf33c9c6d7be8f91d02b7c759c68d8bbe...,23.0,610776024,2,0.008458,0,2
3,5dac2574475af546fd33bd0fc25752878faf5e3a91bd4a...,23.0,610776024,2,0.008458,0,2
4,83366e822f3d1b09270fd862649261a3a8ed63e3217f5e...,25.0,610776024,1,0.008458,1,2


In [45]:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [46]:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [47]:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [48]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,count,sales_channel_id_1,sales_channel_id_2
0,30,25.0,1722,0.008458,1,2,1,0
1,3192,28.0,1722,0.008458,1,2,0,1
2,9586,23.0,1722,0.008458,0,2,0,1
3,4902,23.0,1722,0.008458,0,2,0,1
4,6914,25.0,1722,0.008458,1,2,1,0


In [49]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

0.7011691402968356

### Color of clothing ###
An important factor in the choice of clothing is the color. People tend to have some colors they wear more often than others.

In [50]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered', 'colour_group_code']].copy()
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code
0,30,25.0,9356,1,0.008458,1,9
1,3192,28.0,9356,2,0.008458,1,9
2,9586,23.0,9356,2,0.008458,0,9
3,4902,23.0,9356,2,0.008458,0,9
4,6914,25.0,9356,1,0.008458,1,9


In [51]:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id', 'colour_group_code'])
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2,colour_group_code_-1,colour_group_code_1,colour_group_code_2,...,colour_group_code_72,colour_group_code_73,colour_group_code_80,colour_group_code_81,colour_group_code_82,colour_group_code_83,colour_group_code_90,colour_group_code_91,colour_group_code_92,colour_group_code_93
0,30,25.0,9356,0.008458,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3192,28.0,9356,0.008458,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9586,23.0,9356,0.008458,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4902,23.0,9356,0.008458,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6914,25.0,9356,0.008458,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

0.5144576094296764

### Combination ###

In [53]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered', 'colour_group_code']].copy()
transactions_processed = transactions_processed.merge(recently_sold, how="inner", on='article_id')
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id', 'colour_group_code'])
transactions_processed.head()

,customer_id,age,article_id,price,ordered,count,sales_channel_id_1,sales_channel_id_2,colour_group_code_1,colour_group_code_2,...,colour_group_code_71,colour_group_code_72,colour_group_code_73,colour_group_code_81,colour_group_code_82,colour_group_code_83,colour_group_code_90,colour_group_code_91,colour_group_code_92,colour_group_code_93
0,30,25.0,1722,0.008458,1,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3192,28.0,1722,0.008458,1,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9586,23.0,1722,0.008458,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4902,23.0,1722,0.008458,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6914,25.0,1722,0.008458,1,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

0.7039694763371604

Adding one-hot-encoding for colour here does actually improve the total. Without it, the score becomes worse.